# Vanilla PG
---
* value based 는 Q, V 를 직접적으로 계산하거나 근사했음
    * 이 때 문제점은 TD error (q 러닝 계열) 가 항상 발생하면서 그게 계속 쌓인다는 점
    * 너무 긍정적으로 평가하는데 비해 실제 reward 획득은 그에 훨씬 못 미치는 경우도 많음
    * 그래서 점점 bias 가 되는 문제점
    * policy 를 직접 학습하지 않고 Q 평가를 한다는 점에서 critic 모델이라고 볼 수 있음
* 또 value based 는 discrete 액션만 다룰 수 있음
    * 그리고 deterministic 한 policy 만을 만들어 냄
* policy based
    * p(s|a) 정책을 직접 계산하거나 근사함
    * 즉, 가상의 Q 를 평가하지 않고 액션에 대한 결과를 직접 평가할 수 있음
    * continous 액션 적용할 수 있고, stochastic policy 도 가능함
    * 대신 variance 가 높아짐 - 샘플링 추정을 하는 RL 은 trade off 는 어쩔 수 없이 발생함
    * variance - bias trade off 를 이해하고 총합을 줄이는 방법이 RL agent 를 안정화시키는 핵심 기술임
    * 왜냐면 상호작용이라는 특성 때문에 환경이 조금만 달라져도 급격하게 성능이 무너지는 게 RL 이기 때문
    * RL 을 다룰 땐 항상 근본적인 문제를 생각해야 함 - 불안정, 복잡성
    * trajectory 를 보상 (R, Q, V, A 등) 에 따라 최적화하는 방식 - M.C 로 Q 학습하는 방식과 겉으로는 닮음
    * 더 높은 보상, Q ,A 를 가지는 공간으로 trajectory (s, a, s, a) 를 구겨넣는 방식
    * 여기서 trajectory 는 사실 신경망으로 표현되므로 사실상 traj 를 압축하는 w 를 고차원 공간상에서 업데이트 하는 거라고 생각해야함
* RL 의 근본 개념
    * 사실 RL 의 가장 근원적인 정의를 구현해낸 알고리즘.
    * 아이러니하게 value based 가 먼저 나왔지만 Q 라는 추상적인 평가 함수 (기대 보상 합, reward to go) 로 기술해냄
    * PG 는 보상의 최대화라는 목적을 충실하게 이행함
* PG
    * 일단, on policy 알고리즘
    * episode 끝날 때까지 roll out 을 모두 모으고 끝나면 한 스텝에 대한 정보씩 차례로 싹 업데이트 함
    * 신경망으로 gradient 를 타고 올라가려면, optimizer 가 인식할 수 있는 pseudo loss 가 필요함
    * 그게 J 함수
* contiuous action
    * PG 는 policy 를 직접적으로 모델링하는 알고리즘이기 때문에, 그 어떤 형태의 action 도 유연하게 다룰 수 있음
    * discrete 는 물론이고 continuous action 도 표현가능함
    * 네트워크 출력을 softmax 로 표현하면 discrete action
    * 출력을 어떤 분포 (가우시안) 의 파라미터 (평균, 분산) 으로 표현하게 설계하면 얼마든지 continuous action 표현 가능
    * discrete 표현은 DQN 처럼 e-greedy 방식으로 최종 action 선택할 수 있음
    * categorical action 과 혼용하지만 구분할 수도 있음
    * 여러 개 action 중 k 개를 선택해서 실행하는 구조도 고려할 수 있음
    * 그러나 policy network 의 출력에 softmax 를 통과시켜 확률분포 형태를 나타내지 않으면 deterministic policy 로 봐야함 - 출력값을 그대로 쓰니까
    * continuous action, stochastic policy 는 분포의 매개변수를 출력하는데, 이 매개변수에 대해 그래디언트를 계산하게 됨. J 에는 그 값으로 샘플링한 값을 사용하고 업데이트
    * continuous action, deterministic policy 는 출력값 자체를 그대로 적용하는데, 이 출력 제어값 자체의 그래디언트를 계산하고 업데이트함
    * objective 는 log prob * Q(or R) 형태를 띄는데 두 항 간의 수치 균형을 이루는 것이 안정적인 학습에 중요함
    * 잘못 학습하면 이상한 action 만 뽑게 학습할 수 있음
* action vs policy
    * action - discrete action vs continuous action
    * policy - stochastic policy vs deterministic policy
    * 알고리즘 디자인에 따라 2x2 조합이 모두 가능함
* deterministic policy 는 POMDP 상황에서 특히 문제가 될 수 있음
    * 결국 exploration 차원의 문제
    * stochastic 은 근본적으로 exploration 에 열려 있음
    * stochastic 이지만 deterministic 하게 변하는 경우도 많음
    * 이 때는 표준 편차가 매우 작게 되도록 학습이 되어 뾰족한 분포에서 샘플링을 하게 됨
    * 그런데 또 어떤 문제에서는 50:50 등으로 완전 stochastic 하게 되고 이 때는 꽤 큰 표준 편차를 가질 수도 있음
    * 그래서 뭐라고 단언하기가 어렵고 그래서 RL 은 학습과 학습 결과 해석이 어렵다.
    * 근데 어쨌든 stochastic 한 환경에서는 데이터가 훨씬 더 많이 필요하게 되는게 일반적임
* DPG
    * continuous action + deterministic policy 는 알고리즘 고안이 다소 다름
    * 그러나 이는 stochastic policy 의 특수한 형태라는 것이 증명됨
    * action 선택이 deterministic 하므로 action 에 대해 기댓값 계산이 빠지고 state 에 대한 기댓값 계산만 남음
    * 기본적으로 신경망을 쓰게 되면 on-policy 가 되었을 때 target 이 계속 변하는 문제가 있음
    * deterministic 이면 exploration 이 부족해짐
    * 이런 문제들 때문에 off-policy 방법인 actor-critic 알고리즘을 도입하게 됨
    * 그래도 강제로 exploration 하게 하는 방법 있는듯 - aciton 에 noise 를 추가하는 등
* actor-critic DPG
    * actor loss 는 Q 의 값에 대해서만 정의됨
    * PG theorem 에서는 log_pi 는 확률적 값에 서만 정의되는데 deterministic action 은 확률이 없으므로 그냥 제거함
    * 그리고 critic 에서 나온 Q 값만 가지고 actor 를 업데이트 함
    * 그리고 다시 한 번 말하지만 Q 값은 Q 값이지, 확률 값이 아님 - 확률은 policy 임
    * actor-critic 에서 critic Q 는 기존 value based 와 다른것 같은데 이건 AC 알고리즘 리뷰에서 자세히 다룰 것

* Monte-Carlo 방식 업데이트이므로 variance 가 높다

---
* policy based - 거의 on policy
* value based - 거의 off policy?

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch.autograd import variable
import numpy as np
import gym
import random

In [2]:
env = gym.make('CartPole-v1')

In [17]:
EPISODE = 1000
EPSILON = 1
RATE = .001
GAMMA = .95

In [18]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []
        self.fc1 = nn.Linear(4, 64)
        self.outlayer = nn.Linear(64, 2)
        self.optimizer = optim.Adam(self.parameters(), RATE)
        
    def pi(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.outlayer(x), dim=0)
        return x
    
    def train(self):
        discounted = 0
        for r, log_p in self.data[::-1]:
            discounted = r + GAMMA * discounted
            # loss 정의
            loss = - log_p * discounted
            # 그래디언트 계산
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        self.data = []

In [19]:
state = env.reset()
ep = 1
net = Policy()

while(ep < EPISODE):
    done = False
    total_reward = 0
    
    while(not done):
        #env.render()

        # policy run
        action_prob = net.pi(torch.from_numpy(state).float())
        m = Categorical(action_prob)
        a = m.sample()

        # action 선택
        if(EPSILON < random.randrange(0,1)):
            action = env.action_space.sample()
        else:
            action = a.item()

        # step
        state_next, reward, done, _ = env.step(action)

        # reward 추가
        total_reward += reward
        
        if(done):
            reward = -100

        # data 추가
        net.data.append((reward, torch.log(action_prob[a])))

        # state 갱신
        state = state_next

        # end episode
        if(done):
            print(total_reward)
            total_reward = 0
            ep += 1
            
            # env 초기화
            state = env.reset()
            EPSILON = 1 / (ep / 100 + 1)
            
            # 학습
            net.train()
env.close()

34.0
21.0
12.0
14.0
10.0
8.0
12.0
19.0
30.0
21.0
18.0
62.0
39.0
27.0
19.0
23.0
24.0
16.0
12.0
17.0
15.0
20.0
9.0
41.0
19.0
18.0
24.0
10.0
57.0
11.0
26.0
44.0
31.0
16.0
9.0
26.0
38.0
19.0
18.0
19.0
18.0
27.0
32.0
25.0
37.0
23.0
15.0
39.0
40.0
20.0
21.0
20.0
26.0
33.0
36.0
45.0
30.0
18.0
43.0
13.0
49.0
23.0
43.0
26.0
27.0
25.0
20.0
28.0
15.0
9.0
9.0
25.0
88.0
19.0
39.0
51.0
24.0
49.0
32.0
20.0
34.0
21.0
30.0
66.0
17.0
40.0
11.0
23.0
9.0
28.0
49.0
40.0
46.0
26.0
35.0
20.0
14.0
32.0
23.0
31.0
27.0
72.0
29.0
99.0
78.0
40.0
108.0
88.0
43.0
57.0
46.0
126.0
157.0
70.0
160.0
132.0
81.0
56.0
27.0
90.0
204.0
106.0
145.0
151.0
34.0
103.0
447.0
146.0
96.0
156.0
79.0
49.0
104.0
75.0
240.0
551.0
135.0
112.0
315.0
676.0
75.0
1238.0
137.0
160.0
521.0
440.0
404.0
121.0
593.0
141.0
217.0
388.0
300.0
605.0
570.0
105.0
133.0
307.0
101.0
491.0
127.0
569.0
294.0
594.0
158.0
164.0
640.0
98.0
302.0
2000.0
480.0
447.0
1796.0
216.0
241.0
1179.0
272.0
186.0
273.0
276.0
264.0
203.0
208.0
398.0
237.0
306.0
267.0
27

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque

class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_dim, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, action_dim)
        self.max_action = max_action

    def forward(self, state):
        a = nn.functional.relu(self.fc1(state))
        a = nn.functional.relu(self.fc2(a))
        return self.max_action * torch.tanh(self.fc3(a))

class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_dim + action_dim, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 1)

    def forward(self, state, action):
        q = torch.cat([state, action], 1)
        q = nn.functional.relu(self.fc1(q))
        q = nn.functional.relu(self.fc2(q))
        return self.fc3(q)

class DPG:
    def __init__(self, state_dim, action_dim, max_action):
        self.actor = Actor(state_dim, action_dim, max_action)
        self.actor_optim = optim.Adam(self.actor.parameters(), lr=0.0001)
        self.critic = Critic(state_dim, action_dim)
        self.critic_optim = optim.Adam(self.critic.parameters(), lr=0.001)
        self.replay_buffer = deque(maxlen=1000000)
        self.discount = 0.99
        self.tau = 0.001
        self.batch_size = 128
        self.expl_noise = 0.1
        self.policy_noise = 0.2
        self.noise_clip = 0.5

    def select_action(self, state):
        state = torch.FloatTensor(state.reshape(1, -1))
        action = self.actor(state).cpu().data.numpy().flatten()
        return action + np.random.normal(0, self.expl_noise, size=action.shape)

    def store_experience(self, state, action, reward, next_state):
        self.replay_buffer.append((state, action, reward, next_state))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        state, action, reward, next_state = zip(*random.sample(self.replay_buffer, self.batch_size))
        state = torch.FloatTensor(np.array(state))
        action = torch.FloatTensor(np.array(action))
        reward = torch.FloatTensor(np.array(reward)).reshape(-1, 1)
        next_state = torch.FloatTensor(np.array(next_state))

        target_q = reward + self.discount * self.critic(next_state, self.actor(next_state)).detach()
        critic_loss = nn.MSELoss()(self.critic(state, action), target_q)

        self.critic_optim.zero_grad()
        critic_loss.backward()
        self.critic_optim.step()

        actor_loss = -self.critic(state, self.actor(state)).mean()

        self.actor_optim.zero_grad()
        actor_loss.backward()
        self.actor_optim.step()

        for param, target_param in zip(self.actor.parameters(), self.target_actor.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

        for param, target_param in zip(self.critic.parameters(), self.target_critic.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

    def train_target(self):
        for param, target_param in zip(self.actor.parameters(), self.target_actor.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

        for param, target_param in zip(self.critic.parameters(), self.target_critic.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

    def update_expl_noise(self, episode):
        self.expl_noise *= 0.99

    def update_policy_noise(self, episode):
        if episode < 100:
            self.policy_noise = 0.3
        elif episode < 200:
            self.policy_noise = 0.2
        else:
            self.policy_noise = 0.1

    def update_noise_clip(self, episode):
        if episode < 100:
            self.noise_clip = 0.5
        elif episode < 200:
            self.noise_clip = 0.4
        else:
            self.noise_clip = 0.3

    def save_model(self, filename):
        torch.save({
            'actor_state_dict': self.actor.state_dict(),
            'critic_state_dict': self.critic.state_dict(),
            'actor_optim_state_dict': self.actor_optim.state_dict(),
            'critic_optim_state_dict': self.critic_optim.state_dict(),
        }, filename)

    def load_model(self, filename):
        checkpoint = torch.load(filename)
        self.actor.load_state_dict(checkpoint['actor_state_dict'])
        self.critic.load_state_dict(checkpoint['critic_state_dict'])
        self.actor_optim.load_state_dict(checkpoint['actor_optim_state_dict'])
        self.critic_optim.load_state_dict(checkpoint['critic_optim_state_dict'])


if __name__ == 'main':
    env = gym.make('Pendulum-v0')
    state_dim = env.observation
    action_dim = env.action_space.shape[0]
    max_action = env.action_space.high[0]
    print(f'State dimension: {state_dim}, Action dimension: {action_dim}')

    agent = DPG(state_dim, action_dim, max_action)

    n_episodes = 100

    for i in range(n_episodes):
        state = env.reset()
        episode_reward = 0
        done = False

        while not done:
            action = agent.select_action(state)
            next_state, reward, done, _ = env.step(action)
            agent.store_experience(state, action, reward, next_state)
            state = next_state
            episode_reward += reward

            agent.train()

        agent.update_expl_noise(i)
        agent.update_policy_noise(i)
        agent.update_noise_clip(i)
        agent.train_target()

        print(f'Episode {i}: Total reward = {episode_reward:.2f}')

        if i % 10 == 0:
            agent.save_model(f'td3_model_episode_{i}.pt')

    env.close()
